In [ ]:
import os

import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2 as cv
import json

In [ ]:
from object_detection.yolo_object_detection import object_detection

In [ ]:
from activity_recognition.sequential.model import LSTMActionClassifier
from activity_recognition.non_sequential.model import MLP
from activity_recognition.combine import Ensemble

In [ ]:
!pip install gpt_index

In [ ]:
from chat.chat import create_answer

In [ ]:
non_seq_model = MLP(858, 8)
seq_model = LSTMActionClassifier(1000, 3, 512, 8)

state_dict_non_seq = torch.load("activity_recognition/with_hl_v2_non_seq_model_params.pth", map_location=torch.device('cpu'))
state_dict_seq = torch.load("activity_recognition/seq_model_params.pth", map_location=torch.device('cpu'))

non_seq_model.load_state_dict(state_dict_non_seq)
seq_model.load_state_dict(state_dict_seq)

In [ ]:
ensemble = Ensemble(seq_model, non_seq_model)

In [ ]:
action = "drill"
root_dir = os.path.join("activity_recognition", "test data", action)
print(root_dir)

In [ ]:
img_dir = os.path.join(os.getcwd(), "activity_recognition", "test data", action)
model_dir = os.path.join(os.getcwd(), "object_detection", "weights", "best.pt")
conf = 0.5
data = object_detection(img_dir=img_dir, model_dir=model_dir, conf=conf, save=True)
print(data)

In [ ]:
# show example yolo output
yolo_out_dir = os.path.join("furniture_assembly", "predict4")
img = cv.cvtColor(cv.imread(os.path.join(yolo_out_dir, "003.jpg")), cv.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

In [ ]:
# Hand landmarks detection

# data = {'file_name': '00043.jpg', 'bb_info': [[655, 260, 212, 250], [310, 158, 159, 123], [484, 135, 285, 106], [503, 306, 698, 268], [94, 357, 188, 163], [725, 344, 239, 271], [709, 291, 287, 272], [64, 433, 128, 91], [121, 300, 103, 308]], 'object_classes': [5, 4, 0, 1, 0, 3, 1, 2, 0]}

import hand_landmarks_detection.detect_hand_landmarks as hl
img_path = os.path.join(img_dir, "003.jpg")

hl_data = hl.detect(cv.imread(img_path), data[2]) # data[1]

In [1]:
# Transformer network for activity recognition
from transformer.inference_transformer import Inference_Transformer

# Change these configurations according to loaded model. Config can be found in transformer/Hyperparameter Tuning/...
num_classes = 8
patch_size = 32
embed_dim = 256
num_heads = 2
num_layers = 1

model_path = "transformer/best_transformer.pth"  # Replace with the model path. Models in transformer/Hyperparameter Tuning/...
class_labels = ['assemble_leg', 'drop_drill', 'drop_screw_driver', 'grab_drill', 'take_leg', 'take_screw_driver', 'use_drill', 'use_screw_driver']
image_path = "transformer/Test Data/41.jpg"  # Replace with the image path

#inference_transformer = Inference_Transformer(model_path, num_classes, embed_dim, num_heads, num_layers, class_labels)
inference_transformer = Inference_Transformer(model_path, num_classes, embed_dim, num_heads, num_layers, class_labels, patch_size)

predicted_class = inference_transformer.predict(image_path)
print(f"Predicted Action: {predicted_class}")

Predicted Action: grab_drill


In [ ]:
# For hand landmark detection, check activity_recognition/non_sequential/data_non_seq.py

predictions = ensemble.predict(root_dir, data, prob=True)

In [ ]:
ensemble.vote()

In [ ]:
vector_index = os.path.join("chat", "data", "vector_index.json")
response = create_answer(vector_index, ensemble.vote())
print(response)